In [ ]:
# | default_exp core

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

from fastcore.test import *
from fastcore.imports import *

import jupyter_black
import nbdev
import nbdev.showdoc as showdoc

jupyter_black.load()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# | export
from copy import deepcopy
from textwrap import indent
from collections import ChainMap
from typing import Callable, Any, Generator, Sequence, Optional

# Core

> Core pieces needed to use during code generation.

## Data

We want a code generator, but before, we need a class to hold data while doing code generation. For this reason, the first class we are developing is creatively named `Data`

We'll see later that our nodes/elements will be nested, creating a parent-children. Instead of backing that in, we'll create that with a mixin, because we'll need it for other purposes later.

In [ ]:
# | export
class WithChildrenMixin:
    """
    Adds `parent`/`children` functionality to a class.
    """

    def __init__(self) -> None:
        self.parent: "WithChildrenMixin" | None = None
        self.children: Sequence["WithChildrenMixin"] = []

    def __len__(self) -> int:
        return len(self.children)

    def __contains__(self, child: "WithChildrenMixin") -> bool:
        return child in self.children

    def append(self, child: "WithChildrenMixin") -> "WithChildrenMixin":
        """
        Add a child element to the children list and set its parent to self.
        """
        self.children.append(child) # type: ignore[attr-defined]
        child.set_parent(self)
        return child

    def set_parent(self, parent: "WithChildrenMixin") -> None:
        """
        Set the parent element of self.
        """
        self.parent = parent

    def __iter__(self) -> Generator:
        def iter_data(obj: "WithChildrenMixin", level: int | None = 0) -> Generator:
            """Simply yields parent and then children"""
            yield obj, level
            for child in obj.children:
                yield from iter_data(child, level=(level or 0) + 1)

        return iter_data(self)

So now, we can make the class to hold the data

In [ ]:
# | export
class Data(WithChildrenMixin):
    """
    Data holder used during code generation. Logic is kept as separate functions.
    """

    parent: Optional["Data"]
    children: Sequence["Data"]

    def __init__(
        self,
        name: str,  # Name of this element
        attrs: dict[str, Any] | None = None,  # Attributes for this element
    ) -> None:
        """
        Initialize Data object.

        """

        self.name = name

        if attrs is None:
            attrs = {}
        self._attrs = attrs

        super().__init__()

    @property
    def attrs(self) -> ChainMap:
        """
        Get the attributes for this element, merged with
        parent's attributes, if available.
        """
        if self.parent:
            return ChainMap(self._attrs, self.parent.attrs)
        return ChainMap(self._attrs)

    def clone(self) -> "Data":
        """
        Create a deep copy of this Data object.

        """
        return deepcopy(self)

    def __eq__(self, a: Any) -> bool:
        """
        Compare this Data object with another object for equality.

        """
        print("==")
        same_name: bool = self.name == a.name
        same_attrs: bool = self.attrs == a.attrs
        same_children: bool = self.children == a.children
        return same_name and same_attrs and same_children

    def __str__(self) -> str:
        """
        Get the string representation of this Data object.

        """
        is_self_closing = not self.children

        if self.children:
            children = "\n".join(map(str, self.children))
            children = children.strip()
            children = f"\n{children}\n"
            children = indent(children, "    ")

        if self.attrs:
            if is_self_closing:
                return f"<{self.name} {dict(self.attrs)} />"
            else:
                return f"<{self.name} {dict(self.attrs)}>{children}</{self.name}>"

        if is_self_closing:
            return f"<{self.name} />"
        else:
            return f"<{self.name}>{children}</{self.name}>"

    __repr__ = __str__

In [ ]:
# | hide
showdoc.show_doc(Data)

---

[source](https://github.com/mintyPT/sal/blob/main/sal/core.py#L52){target="_blank" style="float:right; font-size:smaller"}

### Data

>      Data (name:str, attrs:dict[str,typing.Any]|None=None)

Data holder used during code generation. Logic is kept as separate functions.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| name | str |  | Name of this element |
| attrs | dict[str, typing.Any] \| None | None | Attributes for this element |

### Basic operations

Now that we have a class to hold our data, and only for now, we declare structures to use for code generation, manually. Later, we'll create some data-loaders.

In [ ]:
james = Data("person", {"name": "james"})

print(james)

# ---
test_eq(james.name, "person")
test_eq(james.attrs, {"name": "james"})

<person {'name': 'james'} />


We can add children (note: a child's attributes will also include those of his parent)

In [ ]:
james = Data("person", {"name": "james"})
olive = james.append(Data("person", {"name": "olive"}))
silva = james.append(Data("person", {"name": "silva"}))

andrew = olive.append(Data("person", {"name": "andrew"}))

john = james.append(Data("person", {"name": "john"}))
jane = olive.append(Data("person", {"name": "jane"}))
noname = olive.append(Data("person"))

# ---
test_eq(james.children[0].attrs["name"], "olive")

and a child will know its parent

In [ ]:
print(olive.parent.attrs["name"])

# ---
assert james == olive.parent
# test_eq(james, olive.parent)

james
==


To check the number of children, simply use `len`

In [ ]:
len(james)

# ---
assert len(james) == 3

You can compare elements but they are tested based on their attributes and children

In [ ]:
b = Data("b", {"age": 22})
c = Data("b", {"age": 22})
d = Data("d")
b.append(d)
c.append(d)

# ---
assert b == c
# test_eq(b, c)
assert Data("b", {"name": "santos"}) == Data("b", {"name": "santos"})
# test_eq(Data("b", {"name": "santos"}), Data("b", {"name": "santos"}))
assert Data("b") != Data("c")
# test_ne(Data("b"), Data("c"))
assert Data("b", {"name": "silva"}) != Data("b", {"name": "santos"})
# test_ne(Data("b", {"name": "silva"}), Data("b", {"name": "santos"}))

==
==
==
==


You can test if an element is a child of another

In [ ]:
test_eq(olive in james, True)

### Cloning

You can duplicate any `Data` instance

In [ ]:
james.clone()

<person {'name': 'james'}>
    <person {'name': 'olive'}>
        <person {'name': 'andrew'} />
        <person {'name': 'jane'} />
        <person {'name': 'olive'} />
    </person>
    <person {'name': 'silva'} />
    <person {'name': 'john'} />
</person>

In [ ]:
# | hide

root = Data("root")
root.append(Data("child"))

boot = root
assert root == boot

root.attrs["extra"] = "please"
assert root == boot


shoe = root.clone()
shoe.attrs["extra"] = "please2"
assert root != shoe

==
==
==
==


### Logic

#### Basic iteration of all the elements

If you just need to iterate through all the elements, a simple loop will suffice

In [ ]:
for person, level in james:
    print("   " * level, person.name + "::" + person.attrs["name"])

 person::james
    person::olive
       person::andrew
       person::jane
       person::olive
    person::silva
    person::john


#### Map

To do a `map-like` sequencial processing on data and transform it to anything else, use `map_data` which will return instances of `MappedData`

In [ ]:
# | export
class MappedData(WithChildrenMixin):
    """Data structure used to return results from the `map_data` function"""

    def __init__(self, value: Any):
        self.value = value
        super().__init__()

In [ ]:
# | export
def map_data(obj: Data, process: Callable, level: int|None=0) -> MappedData:
    """Maps over a `Data` inst returning `MappedData` instances"""
    child_results = [map_data(c, process, level=(level or 0) + 1) for c in obj.children]
    value = process(obj, level)
    data = MappedData(value)
    for c in child_results:
        data.append(c)
    return data

Here's an example showing how to process all the `Data` instances in a tree...

In [ ]:
def fun(data: Data, level: int = 0) -> str:
    name = data.attrs["name"]
    level_str = " " * (level + 1)
    return f"{level_str}-I'm {name}-"


result = map_data(james.clone(), fun)

...and print the result

In [ ]:
for obj, level in result:
    print(obj.value)

 -I'm james-
  -I'm olive-
   -I'm andrew-
   -I'm jane-
   -I'm olive-
  -I'm silva-
  -I'm john-


In [ ]:
# | hide
test_eq(result.children[0].value, "  -I'm olive-")

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()